In [2]:
import pandas as pd
import ast
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import swifter

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [161]:
def preprocess(text):
    stopW = stopwords.words('english')

    # To stem
    ps = PorterStemmer()

    # Tokenize

    clean_transcript = text.lower()
    clean_transcript = word_tokenize(clean_transcript)
    # Remove digits
    clean_transcript = [i for i in clean_transcript if not re.match(r'\d+', i)]
    # Remove Stopwords and single characters
    clean_transcript = [i for i in clean_transcript if i not in stopW and len(i) > 1]
    # Stemming
    clean_transcript = [ps.stem(word) for word in clean_transcript]

    # Convert back to a string
    clean_transcript = " ".join(clean_transcript)
    
    return clean_transcript

def get_bigram_count(text):
    stopW = stopwords.words('english')

    # To stem
    ps = PorterStemmer()

    # Tokenize

    clean_transcript = text.lower()
    clean_transcript = word_tokenize(clean_transcript)
    # Remove digits
    clean_transcript = [i for i in clean_transcript if not re.match(r'\d+', i)]
    # Remove Stopwords and single characters
    clean_transcript = [i for i in clean_transcript if i not in stopW and len(i) > 1]
    # Stemming
    clean_transcript = [ps.stem(word) for word in clean_transcript]
    
    return len(clean_transcript) - 1

def get_1200_bigram_count(text):
    stopW = stopwords.words('english')

    # To stem
    ps = PorterStemmer()

    # Tokenize

    clean_transcript = text.lower()
    clean_transcript = word_tokenize(clean_transcript)
    # Remove digits
    clean_transcript = [i for i in clean_transcript if not re.match(r'\d+', i)]
    # Remove Stopwords and single characters
    clean_transcript = [i for i in clean_transcript if i not in stopW and len(i) > 1]
    # Stemming
    clean_transcript = [ps.stem(word) for word in clean_transcript]
    
    n = len(clean_transcript)
    
    c = 0
    
    for i in range(n-1):
        if (clean_transcript[i] + '.' + clean_transcript[i+1]) in bigrams:
            c+=1
    
    return c

# Config

In [1]:
year = 2016
bbc_id = 54

# NN Topic Shares

In [ ]:
df = pd.read_csv('../data/partition_predictions/topics_pred_on_bert_partitioned_bbc_{}_{}_with_news_classifier_no_Others.csv'.format(bbc_id, year))
df = df.drop(['Unnamed: 0'], axis=1)
df['topic'] = df['topic'].apply(lambda x: ast.literal_eval(x)[0][0])
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].apply(lambda x: x.month)
df['transcript'] = df['transcript'].swifter.apply(lambda x: preprocess(x))
df.columns = ['partition_id', 'month', 'transcript', 'topic']
df

In [ ]:
df['char_length'] = df['transcript'].apply(lambda x: len(x))
df['word_length'] = df['transcript'].apply(lambda x: len(x.split()))
df

In [ ]:
topics = df['topic'].unique()
months = ['jan', 'feb', 'march', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']


char_length_by_topic_month = {month:{topic: 0 for topic in topics} for month in range(1, 13)}
word_length_by_topic_month = {month:{topic: 0 for topic in topics} for month in range(1, 13)}


In [ ]:
topics

In [ ]:
for index, row in df.iterrows():
    month = row['month']
    topic = row['topic']
    char_len = row['char_length']
    word_len = row['word_length']
    char_length_by_topic_month[month][topic] += char_len
    word_length_by_topic_month[month][topic] += word_len

In [ ]:
rows = []

for topic in topics:
    row = [topic]
    for month in range(1, 13):
        row += [char_length_by_topic_month[month][topic]]
    rows.append(row)
    
share_df = pd.DataFrame(rows, columns=['topic'] + months)

for month in months:
    total = share_df[month].sum()
    share_df[month] = share_df[month] / total
    
share_df = share_df.sort_values(by=['topic']).reset_index(drop=True)
share_df

In [ ]:
share_df.to_csv('nn_topic_shares_{}_char_length.csv'.format(year))

In [ ]:
rows = []

for topic in topics:
    row = [topic]
    for month in range(1, 13):
        row += [word_length_by_topic_month[month][topic]]
    rows.append(row)
    
share_df = pd.DataFrame(rows, columns=['topic'] + months)

for month in months:
    total = share_df[month].sum()
    share_df[month] = share_df[month] / total
    
share_df = share_df.sort_values(by=['topic']).reset_index(drop=True)
share_df

In [ ]:
share_df.to_csv('nn_topic_shares_{}_word_count.csv'.format(year))

# Window Topic Shares

In [60]:
year = 2018

In [63]:
if year == 2018:
    df1 = pd.read_csv('../data/partition_predictions/window_topic_predictions_Jan-Apr_{}.csv'.format(year))
    df2 = pd.read_csv('../data/partition_predictions/window_topic_predictions_May-Aug_{}.csv'.format(year))
    df3 = pd.read_csv('../data/partition_predictions/window_topic_predictions_Sep-Dec_{}.csv'.format(year))
    df = pd.concat([df1, df2, df3])
else:
    df1 = pd.read_csv('../data/partition_predictions/window_topic_predictions_Jan-June_{}.csv'.format(year))
    df2 = pd.read_csv('../data/partition_predictions/window_topic_predictions_July-Dec_{}.csv'.format(year))
    df = pd.concat([df1, df2])

if 'Unnamed: 0' in df.columns:
    df = df.drop(['Unnamed: 0'], axis=1)
if 'vector' in df.columns:
    df = df.drop(['vector'], axis=1)
    
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].apply(lambda x: x.month)
df.columns = ['partition_id', 'month', 'transcript', 'topic']
df

,partition_id,month,transcript,topic
0,705,3,Temperatures up to 10 degrees in London and No...,"Parliament, government and politics"
1,705,3,Live at 4:00 on BBC One and BBC iPlayer.,"Parliament, government and politics"
2,705,3,"They’re simple little jobs, 3 chimpanzee could...","Parliament, government and politics"
3,705,3,"Well, my train’s arrived at its destination, b...",Transport
4,705,3,"This year will be a milestone for the man, as ...","Culture, media and sport"
...,...,...,...,...
32853,1582,10,John is one of a group of Hackney families tak...,Education
32854,1582,10,The Duchess of Cambridge has paid a visit to t...,Defence
32855,1582,10,"Well, I’m off with the team — to do some late ...","Parliament, government and politics"
32856,1582,10,"CHEERING AND APPLAUSE Hello, and welcome to A ...","Parliament, government and politics"


In [64]:
df['char_length'] = df['transcript'].apply(lambda x: len(x))
df['word_length'] = df['transcript'].apply(lambda x: len(x.split()))
df

,partition_id,month,transcript,topic,char_length,word_length
0,705,3,Temperatures up to 10 degrees in London and No...,"Parliament, government and politics",2377,438
1,705,3,Live at 4:00 on BBC One and BBC iPlayer.,"Parliament, government and politics",40,9
2,705,3,"They’re simple little jobs, 3 chimpanzee could...","Parliament, government and politics",55,9
3,705,3,"Well, my train’s arrived at its destination, b...",Transport,576,101
4,705,3,"This year will be a milestone for the man, as ...","Culture, media and sport",804,148
...,...,...,...,...,...,...
32853,1582,10,John is one of a group of Hackney families tak...,Education,1442,256
32854,1582,10,The Duchess of Cambridge has paid a visit to t...,Defence,375,66
32855,1582,10,"Well, I’m off with the team — to do some late ...","Parliament, government and politics",6109,1113
32856,1582,10,"CHEERING AND APPLAUSE Hello, and welcome to A ...","Parliament, government and politics",272,48


In [65]:
topics = df['topic'].unique()
months = ['jan', 'feb', 'march', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']


char_length_by_topic_month = {month:{topic: 0 for topic in topics} for month in range(1, 13)}
word_length_by_topic_month = {month:{topic: 0 for topic in topics} for month in range(1, 13)}

In [66]:
for index, row in df.iterrows():
    month = row['month']
    topic = row['topic']
    char_len = row['char_length']
    word_len = row['word_length']
    char_length_by_topic_month[month][topic] += char_len
    word_length_by_topic_month[month][topic] += word_len

In [67]:
rows = []

for topic in topics:
    row = [topic]
    for month in range(1, 13):
        row += [char_length_by_topic_month[month][topic]]
    rows.append(row)
    
share_df = pd.DataFrame(rows, columns=['topic'] + months)

for month in months:
    total = share_df[month].sum()
    share_df[month] = share_df[month] / total
    
share_df = share_df.sort_values(by=['topic']).reset_index(drop=True)
share_df

,topic,jan,feb,march,apr,may,jun,jul,aug,sep,oct,nov,dec
0,"Agriculture, animals, food and rural affairs",0.017037,0.020001,0.026912,0.013257,0.015110,0.019198,0.040086,0.042854,0.025836,0.028687,0.022119,0.020859
1,"Asylum, immigration and nationality",0.001632,0.001524,0.001076,0.025953,0.004002,0.008456,0.005415,0.005275,0.002685,0.003288,0.002791,0.008358
2,"Business, industry and consumers",0.050693,0.037467,0.038721,0.048926,0.043701,0.055927,0.026846,0.041479,0.025905,0.032446,0.042588,0.031041
3,Communities and families,0.011090,0.022501,0.011210,0.012139,0.017211,0.007852,0.004990,0.006954,0.007429,0.010293,0.009159,0.011523
4,"Crime, civil law, justice and rights",0.058675,0.072483,0.038308,0.060111,0.040007,0.043571,0.042677,0.054109,0.058940,0.049307,0.048543,0.031617
5,"Culture, media and sport",0.161270,0.156280,0.169922,0.168215,0.166950,0.138817,0.171872,0.159257,0.142650,0.131484,0.128386,0.140458
6,Defence,0.020977,0.022365,0.049634,0.028719,0.018822,0.016254,0.016340,0.021541,0.022320,0.026784,0.031547,0.014610
7,Economy and finance,0.055700,0.070637,0.037590,0.028028,0.034727,0.044660,0.018066,0.042331,0.043504,0.060974,0.035895,0.043576
8,Education,0.007908,0.022201,0.014886,0.008173,0.011568,0.006526,0.004105,0.010075,0.009882,0.012408,0.011436,0.005832
9,Employment and training,0.003849,0.004248,0.002620,0.002719,0.003459,0.002703,0.002784,0.005185,0.003727,0.012285,0.007402,0.005188


In [68]:
share_df.to_csv('window_topic_shares_{}_char_length.csv'.format(year))

In [69]:
rows = []

for topic in topics:
    row = [topic]
    for month in range(1, 13):
        row += [word_length_by_topic_month[month][topic]]
    rows.append(row)
    
share_df = pd.DataFrame(rows, columns=['topic'] + months)

for month in months:
    total = share_df[month].sum()
    share_df[month] = share_df[month] / total
    
share_df = share_df.sort_values(by=['topic']).reset_index(drop=True)
share_df

,topic,jan,feb,march,apr,may,jun,jul,aug,sep,oct,nov,dec
0,"Agriculture, animals, food and rural affairs",0.017197,0.019896,0.027467,0.013613,0.015471,0.019401,0.040922,0.043733,0.026254,0.029336,0.022190,0.021115
1,"Asylum, immigration and nationality",0.001586,0.001551,0.001051,0.025436,0.004030,0.008256,0.005323,0.005286,0.002650,0.003289,0.002783,0.008176
2,"Business, industry and consumers",0.050450,0.037121,0.038495,0.048846,0.043560,0.055614,0.026715,0.041492,0.025937,0.032507,0.042458,0.030976
3,Communities and families,0.010872,0.022559,0.011087,0.012074,0.017088,0.007925,0.004922,0.006776,0.007432,0.010313,0.009137,0.011594
4,"Crime, civil law, justice and rights",0.058147,0.072374,0.038307,0.060118,0.039942,0.043165,0.042023,0.053457,0.058409,0.049222,0.048297,0.031492
5,"Culture, media and sport",0.163567,0.158096,0.172045,0.169945,0.169256,0.140715,0.174142,0.161762,0.144920,0.133721,0.129320,0.141614
6,Defence,0.020775,0.022039,0.048894,0.028361,0.018698,0.016004,0.016152,0.021311,0.021678,0.026427,0.031046,0.014424
7,Economy and finance,0.055381,0.070816,0.037409,0.028200,0.034743,0.044513,0.017758,0.042312,0.043410,0.061110,0.035561,0.043623
8,Education,0.007843,0.021907,0.014793,0.008203,0.011578,0.006542,0.004083,0.010136,0.009728,0.012356,0.011367,0.005768
9,Employment and training,0.003944,0.004297,0.002656,0.002800,0.003538,0.002725,0.002745,0.005276,0.003746,0.012065,0.007514,0.005283


In [70]:
share_df.to_csv('window_topic_shares_{}_word_count.csv'.format(year))

# Parliament Speech

In [60]:
df = pd.read_csv('./2015-17_speech.csv')

In [61]:
df

,date,speaker,topic,transcript,pred
0,2015-05-18,Sir Gerald Kaufman (in the Chair),Election of Speaker,"Under the provisions of Standing Order No. 1A,...","Parliament, government and politics"
1,2015-05-18,John Bercow (Buckingham) (Ind),Election of Speaker,"Thank you, Sir Gerald —and warmest congratulat...","Parliament, government and politics"
2,2015-05-18,Jacob Rees-Mogg (North East Somerset) (Con),Election of Speaker,"I beg to move, That the right hon. John Simon ...","Parliament, government and politics"
3,2015-05-18,The Prime Minister (Mr David Cameron),Election of Speaker,"On behalf of the whole House, let me congratul...","Parliament, government and politics"
4,2015-05-18,Ms Harriet Harman (Camberwell and Peckham) (Lab),Election of Speaker,I congratulate my right hon. Friend the Member...,"Parliament, government and politics"
...,...,...,...,...,...
119993,2017-06-29,Mr Bone,Kettering General Hospital,I am very grateful to my hon. Friend for givin...,Health services and medicine
119994,2017-06-29,Mr Hollobone,Kettering General Hospital,"As so often in this place, my hon. Friend spea...",Health services and medicine
119995,2017-06-29,"The Minister of State, Department of Health (M...",Kettering General Hospital,"Let me start by congratulating you, Madam Depu...",Health services and medicine
119996,2017-06-29,Tom Pursglove,Kettering General Hospital,"As part of that work in the Department, will t...",Health services and medicine


In [62]:
df['transcript'] = df['transcript'].apply(lambda x: x.lower())
df['bigram_count'] = df['transcript'].apply(lambda x: get_bigram_count(x))

In [63]:
df

,date,speaker,topic,transcript,pred,bigram_count
0,2015-05-18,Sir Gerald Kaufman (in the Chair),Election of Speaker,"under the provisions of standing order no. 1a,...","Parliament, government and politics",15
1,2015-05-18,John Bercow (Buckingham) (Ind),Election of Speaker,"thank you, sir gerald —and warmest congratulat...","Parliament, government and politics",63
2,2015-05-18,Jacob Rees-Mogg (North East Somerset) (Con),Election of Speaker,"i beg to move, that the right hon. john simon ...","Parliament, government and politics",426
3,2015-05-18,The Prime Minister (Mr David Cameron),Election of Speaker,"on behalf of the whole house, let me congratul...","Parliament, government and politics",313
4,2015-05-18,Ms Harriet Harman (Camberwell and Peckham) (Lab),Election of Speaker,i congratulate my right hon. friend the member...,"Parliament, government and politics",209
...,...,...,...,...,...,...
119993,2017-06-29,Mr Bone,Kettering General Hospital,i am very grateful to my hon. friend for givin...,Health services and medicine,44
119994,2017-06-29,Mr Hollobone,Kettering General Hospital,"as so often in this place, my hon. friend spea...",Health services and medicine,440
119995,2017-06-29,"The Minister of State, Department of Health (M...",Kettering General Hospital,"let me start by congratulating you, madam depu...",Health services and medicine,503
119996,2017-06-29,Tom Pursglove,Kettering General Hospital,"as part of that work in the department, will t...",Health services and medicine,29


In [200]:
df['bigram_count'].sum()

358282

In [64]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].apply(lambda x: x.month)

In [65]:
def preprocess_topic(topic):
    if '|' in topic:
        topic = topic.split('|')[0].strip()
    return topic

df['pred'] = df['pred'].apply(lambda x: preprocess_topic(x))

In [66]:
topics = df['pred'].unique()
months = ['jan', 'feb', 'march', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

bigram_count_by_topic_month = {month:{topic: 0 for topic in topics} for month in range(1, 13)}

In [67]:
topics

array(['Parliament, government and politics',
       'Crime, civil law, justice and rights', 'Employment and training',
       'Culture, media and sport', 'Defence', 'International affairs',
       'Agriculture, animals, food and rural affairs',
       'Health services and medicine', 'Economy and finance',
       'Asylum, immigration and nationality', 'European Union',
       'Education', 'Communities and families', 'Energy and environment',
       'Transport', 'Business, industry and consumers',
       'Housing and planning', 'Social services',
       'Science and technology', 'Social security and pensions'],
      dtype=object)

In [68]:
total = df['bigram_count'].sum()
topic_bigram_count = df[['pred', 'bigram_count']].groupby(['pred'], as_index=False).sum()
topic_bigram_count

,pred,bigram_count
0,"Agriculture, animals, food and rural affairs",68499
1,"Asylum, immigration and nationality",47800
2,"Business, industry and consumers",257758
3,Communities and families,336521
4,"Crime, civil law, justice and rights",972731
5,"Culture, media and sport",308445
6,Defence,287181
7,Economy and finance,906037
8,Education,416325
9,Employment and training,362127


In [71]:
topic_bigram_count['bigram_count'] = topic_bigram_count['bigram_count'] / total

In [74]:
topic_bigram_count.columns = ['topic', 'bigram_share']

In [75]:
topic_bigram_count

,topic,bigram_share
0,"Agriculture, animals, food and rural affairs",0.008092
1,"Asylum, immigration and nationality",0.005647
2,"Business, industry and consumers",0.030450
3,Communities and families,0.039755
4,"Crime, civil law, justice and rights",0.114914
5,"Culture, media and sport",0.036438
6,Defence,0.033926
7,Economy and finance,0.107035
8,Education,0.049183
9,Employment and training,0.042780


In [141]:
df2 = pd.DataFrame([['Others', 0]], columns=topic_bigram_count.columns)
topic_bigram_count = topic_bigram_count.append(df2)
topic_bigram_count = topic_bigram_count.sort_values(by=['topic'])
topic_bigram_count

,topic,bigram_share
0,"Agriculture, animals, food and rural affairs",0.008092
1,"Asylum, immigration and nationality",0.005647
2,"Business, industry and consumers",0.030450
3,Communities and families,0.039755
4,"Crime, civil law, justice and rights",0.114914
5,"Culture, media and sport",0.036438
6,Defence,0.033926
7,Economy and finance,0.107035
8,Education,0.049183
9,Employment and training,0.042780


In [143]:
topic_bigram_count = topic_bigram_count.drop_duplicates()

In [84]:
import datetime
import calendar
months = [ ]
start_date = datetime.datetime(2015, 5, 1)
end_date = datetime.datetime(2017, 6, 30)
curr_date = start_date
delta = datetime.timedelta(days=31)

while curr_date < end_date:
    months.append('{}-{}'.format(calendar.month_abbr[curr_date.month].lower(), curr_date.year))
    curr_date += delta

In [85]:
months

['may-2015',
 'jun-2015',
 'jul-2015',
 'aug-2015',
 'sep-2015',
 'oct-2015',
 'nov-2015',
 'dec-2015',
 'jan-2016',
 'feb-2016',
 'mar-2016',
 'apr-2016',
 'may-2016',
 'jun-2016',
 'jul-2016',
 'aug-2016',
 'sep-2016',
 'oct-2016',
 'nov-2016',
 'dec-2016',
 'jan-2017',
 'feb-2017',
 'mar-2017',
 'apr-2017',
 'may-2017',
 'jun-2017']

In [78]:
import pickle
with open('Ngrams_chosen_from_full.pkl', 'rb') as f:
    ngrams = pickle.load(f)

In [123]:
keys = list(ngrams.keys())

keys_new = []

for key in keys:
    if ('54' in key or '389195' in key or '138620' in key) and 'EU' not in key and 'non_EU' not in key and '54.csv' not in key and '389195.csv' not in key and '138620.csv' not in key:
        keys_new.append(key)
keys_new

['may-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'feb-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'jan-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'jun-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'jul-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'mar-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'apr-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'sep-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'nov-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'dec-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'aug-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'oct-2016 389195_Crime_civil_law_justice_and_rights.csv',
 'mar-2014 389195_Crime_civil_law_justice_and_rights.csv',
 'sep-2014 389195_Crime_civil_law_justice_and_rights.csv',
 'oct-2014 389195_Crime_civil_law_justice_and_rights.csv',
 'jul-2014 389195_Crime_civil_law_justice_and_rights.csv',
 'nov-2014 389195_Crime_civil_law_justice_and_rights.csv

In [124]:
rows = []
for key in keys_new:
    rows.append([key, sum(ngrams[key].values())])
    
res_df = pd.DataFrame(rows, columns=['source', 'bigram_count'])
res_df

,source,bigram_count
0,may-2016 389195_Crime_civil_law_justice_and_ri...,37866
1,feb-2016 389195_Crime_civil_law_justice_and_ri...,26512
2,jan-2016 389195_Crime_civil_law_justice_and_ri...,42945
3,jun-2016 389195_Crime_civil_law_justice_and_ri...,55104
4,jul-2016 389195_Crime_civil_law_justice_and_ri...,41385
...,...,...
3287,BBC+News jul-2015 54_Communities_and_families.csv,340
3288,BBC+News feb-2015 54_Communities_and_families.csv,2307
3289,BBC+News jun-2015 54_Communities_and_families.csv,606
3290,BBC+News jan-2015 54_Communities_and_families.csv,1451


In [125]:
def get_topic(source):
    
    if len(source.split()) == 3:
        return source.split()[2][3:-4]
    else:
        return source.split()[1][7:-4]

def get_month(source):
    if len(source.split()) == 3:
        return source.split()[1]
    else:
        return source.split()[0]

def get_id(source):
    if len(source.split()) == 3:
        return source.split()[2][0:2]
    else:
        return source.split()[1][0:6]

res_df['topic'] = res_df['source'].apply(lambda x: get_topic(x))
res_df['month'] = res_df['source'].apply(lambda x: get_month(x))
res_df['id'] = res_df['source'].apply(lambda x: get_id(x))
res_df

,source,bigram_count,topic,month,id
0,may-2016 389195_Crime_civil_law_justice_and_ri...,37866,Crime_civil_law_justice_and_rights,may-2016,389195
1,feb-2016 389195_Crime_civil_law_justice_and_ri...,26512,Crime_civil_law_justice_and_rights,feb-2016,389195
2,jan-2016 389195_Crime_civil_law_justice_and_ri...,42945,Crime_civil_law_justice_and_rights,jan-2016,389195
3,jun-2016 389195_Crime_civil_law_justice_and_ri...,55104,Crime_civil_law_justice_and_rights,jun-2016,389195
4,jul-2016 389195_Crime_civil_law_justice_and_ri...,41385,Crime_civil_law_justice_and_rights,jul-2016,389195
...,...,...,...,...,...
3287,BBC+News jul-2015 54_Communities_and_families.csv,340,Communities_and_families,jul-2015,54
3288,BBC+News feb-2015 54_Communities_and_families.csv,2307,Communities_and_families,feb-2015,54
3289,BBC+News jun-2015 54_Communities_and_families.csv,606,Communities_and_families,jun-2015,54
3290,BBC+News jan-2015 54_Communities_and_families.csv,1451,Communities_and_families,jan-2015,54


In [126]:
res_df = res_df.loc[res_df['month'].isin(months)]
res_df

,source,bigram_count,topic,month,id
0,may-2016 389195_Crime_civil_law_justice_and_ri...,37866,Crime_civil_law_justice_and_rights,may-2016,389195
1,feb-2016 389195_Crime_civil_law_justice_and_ri...,26512,Crime_civil_law_justice_and_rights,feb-2016,389195
2,jan-2016 389195_Crime_civil_law_justice_and_ri...,42945,Crime_civil_law_justice_and_rights,jan-2016,389195
3,jun-2016 389195_Crime_civil_law_justice_and_ri...,55104,Crime_civil_law_justice_and_rights,jun-2016,389195
4,jul-2016 389195_Crime_civil_law_justice_and_ri...,41385,Crime_civil_law_justice_and_rights,jul-2016,389195
...,...,...,...,...,...
3283,BBC+News aug-2015 54_Communities_and_families.csv,1658,Communities_and_families,aug-2015,54
3286,BBC+News nov-2015 54_Communities_and_families.csv,657,Communities_and_families,nov-2015,54
3287,BBC+News jul-2015 54_Communities_and_families.csv,340,Communities_and_families,jul-2015,54
3289,BBC+News jun-2015 54_Communities_and_families.csv,606,Communities_and_families,jun-2015,54


In [131]:
topic_bigram_count2 = res_df[['bigram_count', 'topic', 'id']].groupby(['topic', 'id'], as_index=False).sum()
topic_bigram_count2

,topic,id,bigram_count
0,Agriculture_animals_food_and_rural_affairs,138620,710543
1,Agriculture_animals_food_and_rural_affairs,389195,619347
2,Agriculture_animals_food_and_rural_affairs,54,173093
3,Asylum_immigration_and_nationality,138620,203675
4,Asylum_immigration_and_nationality,389195,57281
5,Asylum_immigration_and_nationality,54,60460
6,Business_industry_and_consumers,138620,2673149
7,Business_industry_and_consumers,389195,2501775
8,Business_industry_and_consumers,54,511562
9,Communities_and_families,138620,340575


In [128]:
total_bigram_count2 = topic_bigram_count2[['id', 'bigram_count']].groupby(['id'], as_index=False).sum()
total_bigram_count2

,id,bigram_count
0,138620,27614321
1,389195,22332780
2,54,10397596


In [201]:
total_bigram_count2.to_csv('total_bigram_count.csv')

In [129]:
total_bigram_count2_dict = {}
for index, row in total_bigram_count2.iterrows():
    total_bigram_count2_dict[row['id']] = row['bigram_count']
    
total_bigram_count2_dict

{'138620': 27614321, '389195': 22332780, '54': 10397596}

In [132]:
topics_map = {
    'Agriculture_animals_food_and_rural_affairs': 'Agriculture, animals, food and rural affairs',
    'Asylum_immigration_and_nationality': 'Asylum, immigration and nationality',
    'Business_industry_and_consumers': 'Business, industry and consumers',
    'Communities_and_families': 'Communities and families',
    'Crime_civil_law_justice_and_rights': 'Crime, civil law, justice and rights',
    'Culture_media_and_sport': 'Culture, media and sport',
    'Defence': 'Defence',
    'Economy_and_finance': 'Economy and finance',
    'Education': 'Education',
    'Employment_and_training': 'Employment and training',
    'Energy_and_environment': 'Energy and environment',
    'European_Union': 'European Union',
    'Health_services_and_medicine': 'Health services and medicine',
    'Housing_and_planning': 'Housing and planning',
    'International_affairs': 'International affairs',
    'Parliament_government_and_politics': 'Parliament, government and politics',
    'Science_and_technology': 'Science and technology',
    'Social_security_and_pensions': 'Social security and pensions',
    'Social_services': 'Social services',
    'Transport': 'Transport',
    'Others': 'Others'
}

topic_bigram_count2['topic'] = topic_bigram_count2['topic'].apply(lambda x: topics_map[x])
topic_bigram_count2

,topic,id,bigram_count
0,"Agriculture, animals, food and rural affairs",138620,710543
1,"Agriculture, animals, food and rural affairs",389195,619347
2,"Agriculture, animals, food and rural affairs",54,173093
3,"Asylum, immigration and nationality",138620,203675
4,"Asylum, immigration and nationality",389195,57281
5,"Asylum, immigration and nationality",54,60460
6,"Business, industry and consumers",138620,2673149
7,"Business, industry and consumers",389195,2501775
8,"Business, industry and consumers",54,511562
9,Communities and families,138620,340575


In [133]:
topics = list(topic_bigram_count2['topic'].unique())
topics

['Agriculture, animals, food and rural affairs',
 'Asylum, immigration and nationality',
 'Business, industry and consumers',
 'Communities and families',
 'Crime, civil law, justice and rights',
 'Culture, media and sport',
 'Defence',
 'Economy and finance',
 'Education',
 'Employment and training',
 'Energy and environment',
 'European Union',
 'Health services and medicine',
 'Housing and planning',
 'International affairs',
 'Others',
 'Parliament, government and politics',
 'Science and technology',
 'Social security and pensions',
 'Social services',
 'Transport']

In [134]:
ids = ['54', '138620', '389195']
shares = {topic: {_id: 0 for _id in ids} for topic in topics}

for index, row in topic_bigram_count2.iterrows():
    topic = row['topic']
    _id = row['id']
    share = row['bigram_count'] / total_bigram_count2_dict[_id]
    shares[topic][_id] = share

In [137]:
rows = []

for topic in topics:
    row = [topic]
    for _id in ids:
        row.append(shares[topic][_id])
    rows.append(row)
res_df = pd.DataFrame(rows, columns=['topic', 'BBC 54', 'Guardian', 'Telegraph'])
res_df

,topic,BBC 54,Guardian,Telegraph
0,"Agriculture, animals, food and rural affairs",0.016647,0.025731,0.027733
1,"Asylum, immigration and nationality",0.005815,0.007376,0.002565
2,"Business, industry and consumers",0.049200,0.096803,0.112023
3,Communities and families,0.018301,0.012333,0.004924
4,"Crime, civil law, justice and rights",0.074113,0.074742,0.045587
5,"Culture, media and sport",0.108952,0.220948,0.260344
6,Defence,0.029533,0.012133,0.019660
7,Economy and finance,0.048935,0.056454,0.114659
8,Education,0.011516,0.029606,0.026112
9,Employment and training,0.003410,0.011026,0.004970


In [145]:
topic_bigram_count = topic_bigram_count.reset_index(drop=True)

In [146]:
res_df = res_df.sort_values(by='topic')
res_df['Parliament'] = topic_bigram_count['bigram_share']
res_df

,topic,BBC 54,Guardian,Telegraph,Parliament
0,"Agriculture, animals, food and rural affairs",0.016647,0.025731,0.027733,0.008092
1,"Asylum, immigration and nationality",0.005815,0.007376,0.002565,0.005647
2,"Business, industry and consumers",0.049200,0.096803,0.112023,0.030450
3,Communities and families,0.018301,0.012333,0.004924,0.039755
4,"Crime, civil law, justice and rights",0.074113,0.074742,0.045587,0.114914
5,"Culture, media and sport",0.108952,0.220948,0.260344,0.036438
6,Defence,0.029533,0.012133,0.019660,0.033926
7,Economy and finance,0.048935,0.056454,0.114659,0.107035
8,Education,0.011516,0.029606,0.026112,0.049183
9,Employment and training,0.003410,0.011026,0.004970,0.042780


In [147]:
res_df.to_csv('bigram_share_15-17.csv')

# 1200 bigram share

In [193]:
bigrams = pd.read_csv('1200_bigrams.csv')['bigrams'].values

In [194]:
rows = []

for key in keys_new:
    count = 0
    for bigr in bigrams:
        if bigr in ngrams[key]:
            count += ngrams[key][bigr]
    rows.append([key, count, sum(ngrams[key].values())])
    
res_df = pd.DataFrame(rows, columns=['source', '1200_bigram_count', 'bigram_count'])
res_df

,source,1200_bigram_count,bigram_count
0,may-2016 389195_Crime_civil_law_justice_and_ri...,872,37866
1,feb-2016 389195_Crime_civil_law_justice_and_ri...,602,26512
2,jan-2016 389195_Crime_civil_law_justice_and_ri...,874,42945
3,jun-2016 389195_Crime_civil_law_justice_and_ri...,1117,55104
4,jul-2016 389195_Crime_civil_law_justice_and_ri...,931,41385
...,...,...,...
3287,BBC+News jul-2015 54_Communities_and_families.csv,11,340
3288,BBC+News feb-2015 54_Communities_and_families.csv,73,2307
3289,BBC+News jun-2015 54_Communities_and_families.csv,34,606
3290,BBC+News jan-2015 54_Communities_and_families.csv,49,1451


In [195]:
def get_topic(source):
    
    if len(source.split()) == 3:
        return source.split()[2][3:-4]
    else:
        return source.split()[1][7:-4]

def get_month(source):
    if len(source.split()) == 3:
        return source.split()[1]
    else:
        return source.split()[0]

def get_id(source):
    if len(source.split()) == 3:
        return source.split()[2][0:2]
    else:
        return source.split()[1][0:6]

res_df['topic'] = res_df['source'].apply(lambda x: get_topic(x))
res_df['month'] = res_df['source'].apply(lambda x: get_month(x))
res_df['id'] = res_df['source'].apply(lambda x: get_id(x))
res_df

,source,1200_bigram_count,bigram_count,topic,month,id
0,may-2016 389195_Crime_civil_law_justice_and_ri...,872,37866,Crime_civil_law_justice_and_rights,may-2016,389195
1,feb-2016 389195_Crime_civil_law_justice_and_ri...,602,26512,Crime_civil_law_justice_and_rights,feb-2016,389195
2,jan-2016 389195_Crime_civil_law_justice_and_ri...,874,42945,Crime_civil_law_justice_and_rights,jan-2016,389195
3,jun-2016 389195_Crime_civil_law_justice_and_ri...,1117,55104,Crime_civil_law_justice_and_rights,jun-2016,389195
4,jul-2016 389195_Crime_civil_law_justice_and_ri...,931,41385,Crime_civil_law_justice_and_rights,jul-2016,389195
...,...,...,...,...,...,...
3287,BBC+News jul-2015 54_Communities_and_families.csv,11,340,Communities_and_families,jul-2015,54
3288,BBC+News feb-2015 54_Communities_and_families.csv,73,2307,Communities_and_families,feb-2015,54
3289,BBC+News jun-2015 54_Communities_and_families.csv,34,606,Communities_and_families,jun-2015,54
3290,BBC+News jan-2015 54_Communities_and_families.csv,49,1451,Communities_and_families,jan-2015,54


In [196]:
topic_bigram_count2 = res_df[['1200_bigram_count', 'bigram_count', 'topic', 'id']].groupby(['topic', 'id'], as_index=False).sum()
topic_bigram_count2

,topic,id,1200_bigram_count,bigram_count
0,Agriculture_animals_food_and_rural_affairs,138620,23881,1446471
1,Agriculture_animals_food_and_rural_affairs,389195,26445,1341949
2,Agriculture_animals_food_and_rural_affairs,54,7527,409113
3,Asylum_immigration_and_nationality,138620,9354,381222
4,Asylum_immigration_and_nationality,389195,3730,127216
...,...,...,...,...
56,Social_services,389195,1130,32943
57,Social_services,54,1175,35533
58,Transport,138620,25676,1222321
59,Transport,389195,43019,2286748


In [197]:
total_bigram_count2 = topic_bigram_count2[['id', '1200_bigram_count', 'bigram_count']].groupby(['id'], as_index=False).sum()
total_bigram_count2

,id,1200_bigram_count,bigram_count
0,138620,1255868,53109400
1,389195,1046803,45269828
2,54,522634,22533753


In [198]:
total_bigram_count2['share'] = total_bigram_count2['1200_bigram_count'] / total_bigram_count2['bigram_count']

In [199]:
total_bigram_count2

,id,1200_bigram_count,bigram_count,share
0,138620,1255868,53109400,0.023647
1,389195,1046803,45269828,0.023124
2,54,522634,22533753,0.023193


In [ ]:
total_bigram_count2.append(pd.DataFrame([['Parliament', 0, ]]))

In [154]:
total_bigram_count2_dict = {}
for index, row in total_bigram_count2.iterrows():
    total_bigram_count2_dict[row['id']] = row['bigram_count']
    
total_bigram_count2_dict

{'138620': 1255868, '389195': 1046803, '54': 522634}

In [155]:
topic_bigram_count2['topic'] = topic_bigram_count2['topic'].apply(lambda x: topics_map[x])
topic_bigram_count2

,topic,id,bigram_count
0,"Agriculture, animals, food and rural affairs",138620,23881
1,"Agriculture, animals, food and rural affairs",389195,26445
2,"Agriculture, animals, food and rural affairs",54,7527
3,"Asylum, immigration and nationality",138620,9354
4,"Asylum, immigration and nationality",389195,3730
...,...,...,...
56,Social services,389195,1130
57,Social services,54,1175
58,Transport,138620,25676
59,Transport,389195,43019


In [192]:
ids = ['54', '138620', '389195']
shares = {topic: {_id: 0 for _id in ids} for topic in topics}

for index, row in topic_bigram_count2.iterrows():
    topic = row['topic']
    _id = row['id']
    share = row['bigram_count']
    shares[topic][_id] = share

KeyError: 'Agriculture_animals_food_and_rural_affairs'

In [158]:
rows = []

for topic in topics:
    row = [topic]
    for _id in ids:
        row.append(shares[topic][_id])
    rows.append(row)
res_df = pd.DataFrame(rows, columns=['topic', 'BBC 54', 'Guardian', 'Telegraph'])
res_df = res_df.sort_values(by=['topic'])
res_df

,topic,BBC 54,Guardian,Telegraph
0,"Agriculture, animals, food and rural affairs",0.014402,0.019016,0.025263
1,"Asylum, immigration and nationality",0.006509,0.007448,0.003563
2,"Business, industry and consumers",0.045041,0.085272,0.105953
3,Communities and families,0.027442,0.030652,0.016666
4,"Crime, civil law, justice and rights",0.070675,0.066853,0.047013
5,"Culture, media and sport",0.098855,0.160723,0.191772
6,Defence,0.027689,0.012121,0.026156
7,Economy and finance,0.067393,0.097370,0.122536
8,Education,0.017948,0.039030,0.039343
9,Employment and training,0.008543,0.021805,0.009636


In [159]:
df = pd.read_csv('./2015-17_speech.csv')

In [163]:
df['transcript'] = df['transcript'].apply(lambda x: x.lower())
df['bigram_count'] = df['transcript'].swifter.apply(lambda x: get_1200_bigram_count(x))

In [181]:
df['bigram_count2'] = df['transcript'].swifter.apply(lambda x: get_bigram_count(x))

In [182]:
df

,date,speaker,topic,transcript,pred,bigram_count,bigram_count2
0,2015-05-18,Sir Gerald Kaufman (in the Chair),Election of Speaker,"under the provisions of standing order no. 1a,...","Parliament, government and politics",1,15
1,2015-05-18,John Bercow (Buckingham) (Ind),Election of Speaker,"thank you, sir gerald —and warmest congratulat...","Parliament, government and politics",1,63
2,2015-05-18,Jacob Rees-Mogg (North East Somerset) (Con),Election of Speaker,"i beg to move, that the right hon. john simon ...","Parliament, government and politics",7,426
3,2015-05-18,The Prime Minister (Mr David Cameron),Election of Speaker,"on behalf of the whole house, let me congratul...","Parliament, government and politics",16,313
4,2015-05-18,Ms Harriet Harman (Camberwell and Peckham) (Lab),Election of Speaker,i congratulate my right hon. friend the member...,"Parliament, government and politics",9,209
...,...,...,...,...,...,...,...
119993,2017-06-29,Mr Bone,Kettering General Hospital,i am very grateful to my hon. friend for givin...,Health services and medicine,1,44
119994,2017-06-29,Mr Hollobone,Kettering General Hospital,"as so often in this place, my hon. friend spea...",Health services and medicine,11,440
119995,2017-06-29,"The Minister of State, Department of Health (M...",Kettering General Hospital,"let me start by congratulating you, madam depu...",Health services and medicine,18,503
119996,2017-06-29,Tom Pursglove,Kettering General Hospital,"as part of that work in the department, will t...",Health services and medicine,1,29


In [183]:
df['bigram_count'].sum() / df['bigram_count2'].sum()

0.042325628374598856

In [164]:
df['pred'] = df['pred'].apply(lambda x: preprocess_topic(x))

In [165]:
total = df['bigram_count'].sum()
topic_bigram_count = df[['pred', 'bigram_count']].groupby(['pred'], as_index=False).sum()
topic_bigram_count

,pred,bigram_count
0,"Agriculture, animals, food and rural affairs",2110
1,"Asylum, immigration and nationality",1762
2,"Business, industry and consumers",11390
3,Communities and families,16940
4,"Crime, civil law, justice and rights",37034
5,"Culture, media and sport",10267
6,Defence,10435
7,Economy and finance,42797
8,Education,18417
9,Employment and training,22414


In [167]:
df2 = pd.DataFrame([['Others', 0]], columns=topic_bigram_count.columns)
topic_bigram_count = topic_bigram_count.append(df2)
topic_bigram_count = topic_bigram_count.sort_values(by=['pred'])
topic_bigram_count

,pred,bigram_count
0,"Agriculture, animals, food and rural affairs",2110
1,"Asylum, immigration and nationality",1762
2,"Business, industry and consumers",11390
3,Communities and families,16940
4,"Crime, civil law, justice and rights",37034
5,"Culture, media and sport",10267
6,Defence,10435
7,Economy and finance,42797
8,Education,18417
9,Employment and training,22414


In [168]:
topic_bigram_count = topic_bigram_count.drop_duplicates()


In [169]:
total = topic_bigram_count['bigram_count'].sum()
topic_bigram_count['bigram_count'] = topic_bigram_count['bigram_count'] / total

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [171]:
topic_bigram_count = topic_bigram_count.reset_index(drop=True)
topic_bigram_count

,pred,bigram_count
0,"Agriculture, animals, food and rural affairs",0.005889
1,"Asylum, immigration and nationality",0.004918
2,"Business, industry and consumers",0.031791
3,Communities and families,0.047281
4,"Crime, civil law, justice and rights",0.103366
5,"Culture, media and sport",0.028656
6,Defence,0.029125
7,Economy and finance,0.119451
8,Education,0.051404
9,Employment and training,0.062560


In [172]:
res_df['Parliament'] = topic_bigram_count['bigram_count']

In [173]:
res_df

,topic,BBC 54,Guardian,Telegraph,Parliament
0,"Agriculture, animals, food and rural affairs",0.014402,0.019016,0.025263,0.005889
1,"Asylum, immigration and nationality",0.006509,0.007448,0.003563,0.004918
2,"Business, industry and consumers",0.045041,0.085272,0.105953,0.031791
3,Communities and families,0.027442,0.030652,0.016666,0.047281
4,"Crime, civil law, justice and rights",0.070675,0.066853,0.047013,0.103366
5,"Culture, media and sport",0.098855,0.160723,0.191772,0.028656
6,Defence,0.027689,0.012121,0.026156,0.029125
7,Economy and finance,0.067393,0.097370,0.122536,0.119451
8,Education,0.017948,0.039030,0.039343,0.051404
9,Employment and training,0.008543,0.021805,0.009636,0.062560


In [174]:
res_df.to_csv('1200_bigram_share_15-17.csv')

In [184]:
total_bigram_count2

,id,1200_bigram_count,bigram_count,share
0,138620,1255868,53109400,0.023647
1,389195,1046803,45269828,0.023124
2,54,522634,22533753,0.023193


In [188]:
rows = []

for index, row in total_bigram_count2.iterrows():
    rows.append([row['id'], row['share']])
rows.append(['Parliament', df['bigram_count'].sum() / df['bigram_count2'].sum()])

In [189]:
res_df = pd.DataFrame(rows, columns=['source', 'share'])

In [190]:
res_df

,source,share
0,138620,0.023647
1,389195,0.023124
2,54,0.023193
3,Parliament,0.042326


In [191]:
res_df.to_csv('1200_bigram_share.csv')

In [202]:
df

,date,speaker,topic,transcript,pred,bigram_count,bigram_count2
0,2015-05-18,Sir Gerald Kaufman (in the Chair),Election of Speaker,"under the provisions of standing order no. 1a,...","Parliament, government and politics",1,15
1,2015-05-18,John Bercow (Buckingham) (Ind),Election of Speaker,"thank you, sir gerald —and warmest congratulat...","Parliament, government and politics",1,63
2,2015-05-18,Jacob Rees-Mogg (North East Somerset) (Con),Election of Speaker,"i beg to move, that the right hon. john simon ...","Parliament, government and politics",7,426
3,2015-05-18,The Prime Minister (Mr David Cameron),Election of Speaker,"on behalf of the whole house, let me congratul...","Parliament, government and politics",16,313
4,2015-05-18,Ms Harriet Harman (Camberwell and Peckham) (Lab),Election of Speaker,i congratulate my right hon. friend the member...,"Parliament, government and politics",9,209
...,...,...,...,...,...,...,...
119993,2017-06-29,Mr Bone,Kettering General Hospital,i am very grateful to my hon. friend for givin...,Health services and medicine,1,44
119994,2017-06-29,Mr Hollobone,Kettering General Hospital,"as so often in this place, my hon. friend spea...",Health services and medicine,11,440
119995,2017-06-29,"The Minister of State, Department of Health (M...",Kettering General Hospital,"let me start by congratulating you, madam depu...",Health services and medicine,18,503
119996,2017-06-29,Tom Pursglove,Kettering General Hospital,"as part of that work in the department, will t...",Health services and medicine,1,29


In [204]:
df['bigram_count2'].sum()

8464895

In [205]:
news_df = pd.read_csv('../data/news_predictions/news_2016_predictions.csv')

In [206]:
news_df

,Unnamed: 0,source_id,source,day,month,year,program_name,transcript,parliament,top1_topic,top1_acc,top2_topic,top2_acc,top3_topic,top3_acc
0,0,163795,Belfast Telegraph,1,1,2016,My pride and joy;\nTrimble delight at record 2...,Having already become Ulster's most-capped pla...,1,"Parliament, government and politics",47.93,Others,11.57,"Culture, media and sport",9.18
1,1,163795,Belfast Telegraph,1,1,2016,SWEDE TALKER;\nIf you don't recognise Alicia V...,"""It's always there,'' says the rising star, of...",1,Others,35.08,"Business, industry and consumers",19.82,"Culture, media and sport",13.38
2,2,163795,Belfast Telegraph,1,1,2016,Ricky Warwick & Damon Johnson,"Most of Warwick's time is spent on the road, e...",1,"Business, industry and consumers",22.21,"Culture, media and sport",15.08,Others,14.46
3,3,163795,Belfast Telegraph,1,1,2016,PICK OF THE WEEK,The five-piece play a multitude of instruments...,1,"Culture, media and sport",81.15,"Parliament, government and politics",8.58,Others,5.41
4,4,163795,Belfast Telegraph,1,1,2016,Folk royalty are on the one road to a great show,The multi-platinum selling High Kings - Finbar...,1,"Parliament, government and politics",32.26,Others,15.07,"Culture, media and sport",8.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863167,1863167,412338,Wales,31,12,2016,New Year's Eve rail delays as services disrupt...,Arriva Trains Wales services are being further...,1,Transport,97.54,"Crime, civil law, justice and rights",0.43,"Parliament, government and politics",0.41
1863168,1863168,412338,Wales,31,12,2016,Has 2016 been all bad? Of course not! And here...,It's the year Britain was torn in two by Brexi...,1,"Culture, media and sport",30.67,Others,20.27,"Parliament, government and politics",13.23
1863169,1863169,412338,Wales,31,12,2016,Advice for driving in fog: when to use your fo...,Fog can significantly impact driving condition...,1,Transport,31.38,Others,15.98,"Parliament, government and politics",12.02
1863170,1863170,412338,Wales,31,12,2016,Driver left with 'serious' injuries following ...,"The single-vehicle road traffic collision, inv...",1,Others,24.51,"Parliament, government and politics",17.96,Transport,15.37


In [207]:
news_df = news_df.loc[news_df.source_id.isin([138620, 389195])]

In [210]:
news_df = news_df.drop(['day', 'month', 'year', 'top1_topic', 'top2_topic', 'top3_topic'], axis=1)

news_df

,source_id,source,program_name,transcript,parliament,top1_acc,top2_acc,top3_acc
656,138620,Guardian,South Africa turn to former captain Graeme Smi...,The Proteas will make at least two changes to ...,1,23.38,17.68,17.64
657,138620,Guardian,Family of officer shot by Raoul Moat take Nort...,PC David Rathband was shot twice in the face a...,1,27.63,26.36,7.49
658,138620,Guardian,Alastair Cook admits risk in selecting Jimmy A...,The 33-year-old Anderson missed the 241-run wi...,1,42.64,9.93,7.69
659,138620,Guardian,Steven Finn flying again as England field most...,"After several years in cricket limbo, trials a...",1,38.97,20.95,6.92
660,138620,Guardian,Saracens' Alex Goode looks to press home Engla...,After last Sunday's 26-16 win at Wasps - the s...,1,23.74,23.60,11.04
...,...,...,...,...,...,...,...,...
1862871,389195,Telegraph,Beatles' first manager Allan Williams dies age 86,"Credited with discovering the band, Mr William...",1,38.42,15.99,15.01
1862872,389195,Telegraph,Homeless man who returned suit to its owner ha...,Within a week of news breaking about the homel...,1,21.56,18.26,13.40
1862873,389195,Telegraph,Expensive NHS scanners are wasted because of s...,Nearly one million more scans could be carried...,1,69.70,8.73,4.38
1862874,389195,Telegraph,Letters: Patients grow sicker and more stresse...,"We can, in theory, telephone at 8am to see if ...",1,16.84,13.17,9.08


In [212]:
news_df = news_df.dropna(subset=['transcript'])
news_df['word length'] = news_df['transcript'].apply(lambda x: len(x.split()))

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [213]:
news_df

,source_id,source,program_name,transcript,parliament,top1_acc,top2_acc,top3_acc,word length
656,138620,Guardian,South Africa turn to former captain Graeme Smi...,The Proteas will make at least two changes to ...,1,23.38,17.68,17.64,627
657,138620,Guardian,Family of officer shot by Raoul Moat take Nort...,PC David Rathband was shot twice in the face a...,1,27.63,26.36,7.49,518
658,138620,Guardian,Alastair Cook admits risk in selecting Jimmy A...,The 33-year-old Anderson missed the 241-run wi...,1,42.64,9.93,7.69,671
659,138620,Guardian,Steven Finn flying again as England field most...,"After several years in cricket limbo, trials a...",1,38.97,20.95,6.92,1011
660,138620,Guardian,Saracens' Alex Goode looks to press home Engla...,After last Sunday's 26-16 win at Wasps - the s...,1,23.74,23.60,11.04,504
...,...,...,...,...,...,...,...,...,...
1862871,389195,Telegraph,Beatles' first manager Allan Williams dies age 86,"Credited with discovering the band, Mr William...",1,38.42,15.99,15.01,271
1862872,389195,Telegraph,Homeless man who returned suit to its owner ha...,Within a week of news breaking about the homel...,1,21.56,18.26,13.40,227
1862873,389195,Telegraph,Expensive NHS scanners are wasted because of s...,Nearly one million more scans could be carried...,1,69.70,8.73,4.38,275
1862874,389195,Telegraph,Letters: Patients grow sicker and more stresse...,"We can, in theory, telephone at 8am to see if ...",1,16.84,13.17,9.08,1787


In [216]:
news_df[['source', 'word length']].groupby(['source'], as_index=False).mean().to_csv('avg_word_length_guardian_telegraph.csv')

In [218]:
news_df[['source', 'word length']].groupby(['source'], as_index=False).sum()

,source,word length
0,Guardian,82199889
1,Telegraph,80466497


In [219]:
df

,date,speaker,topic,transcript,pred,bigram_count,bigram_count2
0,2015-05-18,Sir Gerald Kaufman (in the Chair),Election of Speaker,"under the provisions of standing order no. 1a,...","Parliament, government and politics",1,15
1,2015-05-18,John Bercow (Buckingham) (Ind),Election of Speaker,"thank you, sir gerald —and warmest congratulat...","Parliament, government and politics",1,63
2,2015-05-18,Jacob Rees-Mogg (North East Somerset) (Con),Election of Speaker,"i beg to move, that the right hon. john simon ...","Parliament, government and politics",7,426
3,2015-05-18,The Prime Minister (Mr David Cameron),Election of Speaker,"on behalf of the whole house, let me congratul...","Parliament, government and politics",16,313
4,2015-05-18,Ms Harriet Harman (Camberwell and Peckham) (Lab),Election of Speaker,i congratulate my right hon. friend the member...,"Parliament, government and politics",9,209
...,...,...,...,...,...,...,...
119993,2017-06-29,Mr Bone,Kettering General Hospital,i am very grateful to my hon. friend for givin...,Health services and medicine,1,44
119994,2017-06-29,Mr Hollobone,Kettering General Hospital,"as so often in this place, my hon. friend spea...",Health services and medicine,11,440
119995,2017-06-29,"The Minister of State, Department of Health (M...",Kettering General Hospital,"let me start by congratulating you, madam depu...",Health services and medicine,18,503
119996,2017-06-29,Tom Pursglove,Kettering General Hospital,"as part of that work in the department, will t...",Health services and medicine,1,29


In [220]:
df['word count'] = df['transcript'].apply(lambda x: len(x.split()))

In [221]:
df['word count'].sum()

17682934